In [1]:
import pandas as pd
import re

# Apenas substitua o nome do arquivo para o que você deseja ler
file_path = 'Matriz horária 2º-2024.xlsx'



def gerar_disciplinas(df):
    # Lista para armazenar as linhas resultantes
    linhas = []
    
    # Verifica se o dataframe possui pelo menos 2 linhas e 2 colunas
    if df.shape[0] < 3 or df.shape[1] < 3:
        print("O dataframe não possui informações suficientes.")
        return pd.DataFrame(linhas)

    # Itera sobre o dataframe, começando da terceira linha
    for i in range(2, df.shape[0]):  # Linha 2 em diante para as disciplinas
        dia_semana = df.iloc[i, 0]  # Dia da semana

        # Itera sobre as colunas, saltando 2 colunas a cada vez
        for col in range(1, df.shape[1], 2):
            if pd.isna(df.iloc[1, col]):  # Verifica se há um horário
                continue
            if col + 1 >= df.shape[1]:  # Verifica se há uma coluna de turma/professor
                break

            horario = df.iloc[1, col]  # Horário da segunda linha
            nome_uc = df.iloc[i, col]  # Nome da UC
            turma_professor = df.iloc[i, col + 1]  # Turma e Professor
            curso_termo = df.iloc[0, col]  # Curso e Termo
            
            # Adiciona a linha se houver informação válida
            if pd.notna(nome_uc) and pd.notna(turma_professor):
                linha = {
                    'Dia da Semana': dia_semana,
                    'Horário': horario,
                    'Nome da UC': nome_uc,
                    'Turma - Professor': turma_professor,
                    'Curso - Termo': curso_termo
                }
                linhas.append(linha)

    # Transforma a lista de dicionários em um DataFrame
    return pd.DataFrame(linhas)

def split_curso_termo(texto):
    partes = re.split(r' \n|\n', texto)
    curso = partes[0]
    termo = re.search(r'Termo (\d+)', partes[1]).group(1) if len(partes) > 1 else None
    return curso, termo

def split_turma_professor(texto):
    # Expressão regular que busca padrões como "NA - Arlindo" ou "N - João"
    match = re.match(r'([A-Z]+[A-Z0-9]*)\s*-\s*(.+)', texto)
    
    if match:
        turma = match.group(1).strip()  # Pega o código da turma (como "NA" ou "IB")
        professor = match.group(2).strip()  # Pega o nome do professor
    else:
        # Caso não haja um separador claro, tenta adivinhar o que é turma e professor
        if texto.strip().isalpha():
            turma = 'I'  # Se o texto tiver só letras (nome do professor), assume "I" como padrão
        else:
            turma = re.search(r'[A-Z]+$', texto).group(0) if re.search(r'[A-Z]+$', texto) else 'N'
        professor = texto.strip()  # Nome do professor sem alterações
    
    return turma, professor

def normalizar_horario(horario):
    # Expressão regular para capturar os horários
    padrao_horario = re.compile(r"(\d{1,2})[hH](\d{2})\s*-\s*(\d{1,2})[hH](\d{2})")
    match = padrao_horario.match(horario)
    
    if match:
        hora_inicio = int(match.group(1))
        minuto_inicio = match.group(2)
        hora_fim = int(match.group(3))
        minuto_fim = match.group(4)

        # Garantir que as horas tenham sempre dois dígitos
        hora_inicio = f"{hora_inicio:02d}"
        hora_fim = f"{hora_fim:02d}"

        # Retorna o horário no formato correto
        return f"{hora_inicio}h{minuto_inicio} - {hora_fim}h{minuto_fim}"
    
    # Retorna o horário original se não corresponder ao padrão
    return horario

def ler_matriz (endereco):
    # Lendo o arquivo
    df = pd.read_excel(endereco)

    # Aprimorando a matriz para facilitar a leitura
    df['Unnamed: 0'] = df['Unnamed: 0'].fillna(method='ffill')
    df = df.T
    df[0] = df[0].fillna(method='ffill')
    df = gerar_disciplinas(df)

    # Limpando dados
    df[['Curso', 'Termo']] = df['Curso - Termo'].str.replace('\n\n\n\n', '').apply(split_curso_termo).apply(pd.Series)
    df[['Turma', 'Professor']] = df['Turma - Professor'].apply(split_turma_professor).apply(pd.Series)
    df['Horário'] = df['Horário'].apply(normalizar_horario)
    df.update(df.loc[:, df.columns != 'ID'].apply(lambda x: x.str.strip() if x.dtype == "object" else x))

    # Renomeando com o padrão atual
    df['ID'] = df.index
    df = df[['ID', 'Nome da UC', 'Professor', 'Turma', 'Horário', 'Dia da Semana', 'Curso', 'Termo']]
    df.columns = ['ID', 'NOME', 'PROFESSORES', 'TURMA', 'HORARIO', 'DIA', 'CURSO', 'TERMO']

    return df

In [2]:
df = ler_matriz(file_path)
df

C:\Users\Atual\AppData\Local\Temp\ipykernel_15068\2748217045.py:97: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df['Unnamed: 0'] = df['Unnamed: 0'].fillna(method='ffill')
C:\Users\Atual\AppData\Local\Temp\ipykernel_15068\2748217045.py:99: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df[0] = df[0].fillna(method='ffill')


,ID,NOME,PROFESSORES,TURMA,HORARIO,DIA,CURSO,TERMO
0,0,Matemática Geral,Samia,I,08h00 - 10h00,Segunda,BCT-I,2
1,1,Matemática Discreta,Renato,IA,10h00 - 12h00,Segunda,BCT-I,2
2,2,Matemática Discreta,Renato,IB,13h30 - 15h30,Segunda,BCT-I,2
3,3,Matemática Discreta,Grasiele,IC,15h30 - 17h30,Segunda,BCT-I,2
4,4,Matemática Discreta,Pedro,NA,19h00 - 21h00,Segunda,BCT-N,2
...,...,...,...,...,...,...,...,...
404,404,TCC II - EC,Cappabianco,I,15h30 - 17h30,Sábado,BCC/EC-N,10
405,405,TCC II,Elisa,I,08h00 - 10h00,Sábado,BBT-I,8
406,406,TCC II,Elisa,I,10h00 - 12h00,Sábado,BBT-I,8
407,407,Estágio,Elisabeth,I,13h30 - 15h30,Sábado,BBT-I,8


In [5]:
import pandas as pd
import json

# Função para combinar os horários
def combinar_horarios(df):
    # Agrupa pelos campos NOME, PROFESSORES e TURMA
    df_agrupado = df.groupby(['NOME', 'PROFESSORES', 'TURMA']).agg({
        'HORARIO': lambda x: list(x),  # Converte os horários em uma lista
        'DIA': lambda x: list(x),      # Converte os dias em uma lista
        'CURSO': 'first',              # Pega o primeiro valor de CURSO
        'TERMO': 'first'               # Pega o primeiro valor de TERMO
    }).reset_index()
    
    return df_agrupado

# Função para converter o DataFrame agrupado em JSON e salvar
def salvar_json(df_agrupado):
    file_path='disciplinas.json'
    df_agrupado['ID'] = df_agrupado.index
    
    # Converte o DataFrame para uma lista de dicionários
    resultado = df_agrupado.to_dict(orient='records')
    
    # Salva em um arquivo JSON
    with open(file_path, 'w', encoding='utf-8') as f:
        json.dump(resultado, f, ensure_ascii=False, indent=4)
    
    print(f"Arquivo JSON salvo em {file_path}")

# Combina os horários e salva o arquivo
def combinar_salvar(df):
    df_agrupado = combinar_horarios(df)
    salvar_json(df_agrupado)
    return df_agrupado

In [6]:
combinar_salvar(df)

Arquivo JSON salvo em disciplinas.json


,NOME,PROFESSORES,TURMA,HORARIO,DIA,CURSO,TERMO,ID
0,Algoritmos e Estruturas de Dados I,Arlindo,IA,"[13h30 - 15h30, 13h30 - 15h30]","[Terça, Quinta]",BCT-I,2,0
1,Algoritmos e Estruturas de Dados I,Arlindo,NA,"[19h00 - 21h00, 19h00 - 21h00]","[Terça, Quinta]",BCT-N,2,1
2,Algoritmos e Estruturas de Dados I,Didier,IB,"[10h00 - 12h00, 10h00 - 12h00]","[Quarta, Sexta]",BCT-I,2,2
3,Algoritmos e Estruturas de Dados I,Luis,NB,"[21h00 - 23h00, 21h00 - 23h00]","[Segunda, Quarta]",BCT-N,2,3
4,Algoritmos e Estruturas de Dados II,Berton,I,"[13h30 - 15h30, 13h30 - 15h30]","[Terça, Quinta]",BCT-I,4,4
...,...,...,...,...,...,...,...,...
227,Tópicos especiais em Economia e Negócios I,Scriptore,I,[13h30 - 15h30],[Sexta],BCT-I,4,227
228,Tópicos especiais em Economia e Negócios I,Scriptore,N,[21h00 - 23h00],[Sexta],BCT-N,4,228
229,"Vidros, Vitrocerâmicos e Vidrados",Eliandra,I,"[15h30 - 17h30, 08h00 - 10h00, 10h00 - 12h00]","[Quarta, Sexta, Sexta]",EB-I,8,229
230,Álgebra Linear Computacional,Vanessa,I,"[15h30 - 17h30, 15h30 - 17h30]","[Terça, Quinta]",BCT-I,6,230
